In [ ]:
from pathlib import Path
import os
import sys

current_path = Path.cwd()
if current_path.name == "notebooks":
    project_root = current_path.parent
else:
    project_root = current_path

os.chdir(project_root)
print(f"Current working directory: {Path.cwd()}")

sys.path.append(str(project_root / "src"))


# 04 - Modelado Supervisado

## Objetivo
Entrenamiento y evaluación de modelos para predecir precios.

## 1. División de datos (train/test)

## 2. Modelo base: Regresión Lineal (GLM)

## 3. Modelo complejo: Gradient Boosting (GBM)

## 4. Comparación de métricas

## 5. Interpretación con SHAP y/o LIME